# EigenCentrality

We have a network with N nodes in it. N by N Matrix $X$ describes strengths of the connections between nodes.
Our goal is to define some notion of "centrality" or "importance" of a node in this network.

Idea: "central" nodes are strongly connected with other "central" nodes. Denote centrality of $i$th node as $v_i$:

$$ v_i = \frac{1}{\lambda} \sum_{j = 1}^N{X_{ij} v_j} $$

Pack all $v_i$ in a single vector $\mathbf{v}$ and write the above line with matrix notation:

$$ \mathbf{v} = \frac{1}{\lambda} X \mathbf{v} $$

$$ X \mathbf{v} = \lambda \mathbf{v} $$

Eigen equation!
However, we know it has many solutions. But only non-negative centralities make sense. 

*Perron-Frobenius Theorem*: If all elements of matrix $X$ are non-negative and there is a path from any node to any node, then there is only one eigenvector with all real non-negative components and it corresponds to the highest by absolute value real positive eigenvector.


# Example: Slice of Six Degrees of Francis Bacon Dataset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import csv
%matplotlib notebook
plt.rcParams['figure.dpi'] = 150

In [ ]:
with open('quakers_nodelist.csv', 'r') as nodecsv: # Open the file
    nodereader = csv.reader(nodecsv) # Read the csv
    # Retrieve the data 
    nodes = dict([(n[0], n[1:]) for n in nodereader][1:])
    
ids = dict([(name, i) for (i, name) in enumerate(nodes.keys())])
ids_names = dict(enumerate(nodes.keys()))
N = len(ids)

with open('quakers_edgelist.csv', 'r') as edgecsv: # Open the file
    edgereader = csv.reader(edgecsv) # Read the csv
    edges = [tuple(e) for e in edgereader][1:] # Retrieve the data

In [ ]:
print(list(nodes.keys())[:10])
print(edges[:10])
print(N)
print(len(edges))

In [ ]:
X = np.zeros((N, N))

for (p1, p2) in edges:
    id1 = ids[p1]
    id2 = ids[p2]
    
    X[id1, id2] = 1.
    X[id2, id1] = 1.
    
X.max(axis=1) # Each person has at least one connection

In [ ]:
eigvals, eigvecs = np.linalg.eig(X)
sorted_idx = np.argsort(eigvals)[::-1]
eigvals, eigvecs = np.real(eigvals), np.real(eigvecs)

    
eigvals, eigvecs = eigvals[sorted_idx], eigvecs[:,sorted_idx]
eigvals[:10]

In [ ]:
eigvecs[:, 0]

In [ ]:
epsilon = 1e-5
off_cluster_ids = np.array(range(N))[np.abs(eigvecs[:, 0]) < epsilon]
off_cluster_ids

In [ ]:
print(ids_names[1], nodes[ids_names[1]])
print(X[1])
print(ids_names[4], nodes[ids_names[4]])
print(X[4])

In [ ]:
most_popular_ids = np.argsort(eigvecs[:, 0])[::-1]
print(ids_names[most_popular_ids[0]], nodes[ids_names[most_popular_ids[0]]])
print(ids_names[most_popular_ids[1]], nodes[ids_names[most_popular_ids[1]]])
print(ids_names[most_popular_ids[2]], nodes[ids_names[most_popular_ids[2]]])
print(ids_names[most_popular_ids[3]], nodes[ids_names[most_popular_ids[3]]])

# Spectral Clustering

If a network is divided into disconnected (or weakly connected) clusters, we can identify them.

Several definitions: degree matrix $D$ of $X$ is a diagonal matrix with $D_{ii}$ being a degree of $i$th node:

$$ D_{ii} = \sum_j{X_{ij}} $$

Laplacian of $X$ is the matrix 

$$ L = D - X $$

It has the following property, for any vector $\mathbf{v}$:

$$ \mathbf{v}^T L \mathbf{v} = \frac{1}{2} \sum_{i, j = 1}^N {X_{ij} (v_i - v_j)^2} $$

Obviously, it has $\mathbf{1}$, the vector consisting as all ones, as an eigenvector, which corresponds to zero eigenvalue.

Moreover, if the network has several disconnected components, indicator vectors for each component will also be an eigenvectors corresponding to zero eigenvalue.

Number of connected components is exactly the multiplicity of zeroth eigenvalue.

In [ ]:
with open('spiral.txt', 'r') as csv_file: # Open the file
    data = csv.reader(csv_file, delimiter='\t') # Read the csv
    # Retrieve the data 
    data = np.array([d[:2] for d in data]).astype(np.float)

# number of points
N = len(data)
    
fig = plt.figure()
plt.scatter(data[:, 0], data[:, 1])

In [ ]:
# X is pairwise similarity between points
X = np.zeros((N, N))

for i in range(N):
    for j in range(N):
        X[i, j] = np.exp(-((data[i] - data[j])**2).sum() / 2.)
        
D = np.diag(np.einsum("ii -> i", X))
X = D - X

In [ ]:
eigvals, eigvecs = np.linalg.eig(X)
sorted_idx = np.argsort(np.abs(eigvals))
eigvals, eigvecs = np.real(eigvals), np.real(eigvecs)

eigvals, eigvecs = eigvals[sorted_idx], eigvecs[:,sorted_idx]
print(eigvals[:10])

relevant_eigencomponents = eigvecs[:, :3]
relevant_eigencomponents

In [ ]:
#from sklearn.cluster import KMeans

#kmeans = KMeans(n_clusters=3).fit(relevant_eigencomponents)

labels1 = np.abs(relevant_eigencomponents[:, 0]) > 0.5e-2
labels2 = np.abs(relevant_eigencomponents[:, 1]) > 0.5e-2
labels3 = np.abs(relevant_eigencomponents[:, 2]) > 0.5e-2


fig = plt.figure()
plt.scatter(data[labels1, 0], data[labels1 == 1, 1])
plt.scatter(data[labels2, 0], data[labels2 == 1, 1])
plt.scatter(data[labels3, 0], data[labels3 == 1, 1])